Esta versão: 
    -> corrige os erros todos (ortograficos e gramaticais)

    -> vê a polarity e subjectivity com o textblob

    -> avalia se a frase é negativa ou afirmativa 
            (utiliza a análise do spaCY para procurar palavras de negação como not, n't, never, ...)

    -> avalia se a frase é factual ou uma opinião 
            (usa um modelo pre treinado, https://huggingface.co/lighteternal/fact-or-opinion-xlmr-el)

                Label 0: Opinion/Subjective sentence
                Label 1: Fact/Objective sentence

    -> classifica a emoção 
            (usa um modelo pre treinado, https://huggingface.co/ayoubkirouane/BERT-Emotions-Classifier)
            
            este modelo tem 11 emoções:
                'anger' 
                'anticipation'
                'disgust'
                'fear'
                'joy'
                'love'
                'optimism'
                'pessimism'
                'sadness'
                'surprise'
                'trust'

    -> vê inference times de todas as funções

In [51]:
import time
import nltk
import spacy
import textblob
from textblob import TextBlob
from spellchecker import SpellChecker
from gramformer import Gramformer
from transformers import pipeline

In [52]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading spaCy model...")
    import os
    os.system("python -m spacy download en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhabid\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Ignore warnings

In [53]:
import warnings
warnings.filterwarnings("ignore")

## Setup the decorator

In [54]:

from types import MethodType

def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

## Initialize models 

In [55]:
class TextAnalyzer:
    def __init__(self):
        self.spell_checker = SpellChecker()
        self.gramformer = Gramformer(models=1, use_gpu=False)
        
        # Initialize emotion classifier
        print("Loading emotion classification model...")
        self.emotion_classifier = pipeline("text-classification", 
        model="ayoubkirouane/BERT-Emotions-Classifier", return_all_scores=True)

        # Initialize specialized fact vs. opinion classifier with correct model name
        print("Loading specialized fact-opinion classification model...")
        self.fact_opinion_classifier = pipeline(
            "text-classification",
            model="lighteternal/fact-or-opinion-xlmr-el"
        )
        
        # For storing timing information
        self.inference_times = {}

## Optimized Gramformer for sentence correction

In [56]:
@add_method(TextAnalyzer)
def correct_spelling(self, sentence):
    start_time = time.time()
    words = sentence.split() 
    corrected_words = [self.spell_checker.correction(word) or word for word in words] 
    result = " ".join(corrected_words)
    elapsed_time = time.time() - start_time
    self.inference_times['spell_correction'] = elapsed_time
    return result

@add_method(TextAnalyzer)
def correct_sentence(self, sentence):
    start_time = time.time()
    spelled_corrected = self.correct_spelling(sentence)
    
    # Separate timing for grammar correction
    grammar_start_time = time.time()
    corrected_sentences = self.gramformer.correct(spelled_corrected, max_candidates=1)
    result = next(iter(corrected_sentences), spelled_corrected)
    grammar_elapsed_time = time.time() - grammar_start_time
    
    elapsed_time = time.time() - start_time
    self.inference_times['grammar_correction'] = grammar_elapsed_time
    self.inference_times['total_correction'] = elapsed_time
    return result

## Polarity detection with Spacy

In [57]:
@add_method(TextAnalyzer)
def analyze_sentence_type(self, text):
    """Determine if the sentence is affirmative or negative"""
    start_time = time.time()
    doc = nlp(text)
    
    # Check for negation
    has_negation = any(token.dep_ == 'neg' for token in doc)
    
    # Determine sentence type
    if has_negation:
        sentence_type = "negation"
    else:
        sentence_type = "affirmative"
    
    elapsed_time = time.time() - start_time
    self.inference_times['sentence_type_analysis'] = elapsed_time
    
    return {
        'sentence_type': sentence_type
    }

## Subjectivity detection with a pre-trained model

In [58]:
@add_method(TextAnalyzer)
def classify_fact_opinion(self, text):
    """Classify if the text is a fact or an opinion using a specialized model."""
    start_time = time.time()
    result = self.fact_opinion_classifier(text)[0]
    
    # This model outputs LABEL_0 (opinion) or LABEL_1 (fact)
    # Convert to more readable format
    label_map = {"LABEL_0": "opinion", "LABEL_1": "fact"}
    classification = label_map.get(result['label'], result['label'])
    confidence = result['score']
    
    elapsed_time = time.time() - start_time
    self.inference_times['fact_opinion_classification'] = elapsed_time
    
    return {
        'classification': classification,
        'confidence': confidence
    }

## Emotion classification with DistilBERT

In [59]:
@add_method(TextAnalyzer)
def detect_emotion(self, text):
    """Detects emotions in text using the BERT-Emotions-Classifier model."""
    start_time = time.time()
    emotion_scores = self.emotion_classifier(text)[0]
    sorted_emotions = sorted(emotion_scores, key=lambda x: x['score'], reverse=True)
    top_emotion = sorted_emotions[0]
    all_emotions = {emotion['label']: emotion['score'] for emotion in sorted_emotions}
    
    elapsed_time = time.time() - start_time
    self.inference_times['emotion_detection'] = elapsed_time
    
    return {"emotion": top_emotion['label'], "confidence": top_emotion['score'], "all_emotions": all_emotions}

# Analysis of the given text

In [60]:
@add_method(TextAnalyzer)
def analyze_text(self, text):
    """Complete analysis of the given text."""
    # Reset timing data for new analysis
    self.inference_times = {}
    
    total_start_time = time.time()
    
    original_text = text
    corrected_text = self.correct_sentence(text)
    
    sentence_params = self.analyze_sentence_type(corrected_text)
    emotion_data = self.detect_emotion(corrected_text)
    fact_opinion_data = self.classify_fact_opinion(corrected_text)
    
    total_elapsed_time = time.time() - total_start_time
    self.inference_times['total_analysis'] = total_elapsed_time
    
    result = {
        'original_text': original_text,
        'corrected_text': corrected_text,
        'needs_correction': original_text != corrected_text,
        'sentence_type': sentence_params['sentence_type'],
        'emotion': emotion_data['emotion'],
        'emotion_confidence': emotion_data['confidence'],
        'all_emotions': emotion_data['all_emotions'],
        'fact_opinion': fact_opinion_data['classification'],
        'fact_opinion_confidence': fact_opinion_data['confidence'],
        'inference_times': self.inference_times
    }
    
    return result

## Sentence classification

In [61]:
def main():
    analyzer = TextAnalyzer()
    
    print("Text Analysis Tool")
    print("------------------")
    print("Enter a sentence to analyze (or 'quit' to exit):")
    
    while True:
        user_input = input("\nYour sentence: ")
        if user_input.lower() == 'quit':
            break
        
        result = analyzer.analyze_text(user_input)
        
        print("\nAnalysis Results:")
        print("-----------------")
        if result['needs_correction']:
            print(f"Corrected: {result['corrected_text']}")
            
        print("---")
        print(f"Sentence type: {result['sentence_type']}")
        print(f"Classification: {result['fact_opinion']} (confidence: {result['fact_opinion_confidence']:.2f})")
        print(f"Emotion detected: {result['emotion']} (confidence: {result['emotion_confidence']:.2f})")
        
        print("\nAll detected emotions and their confidence scores:")
        for emotion, score in result['all_emotions'].items():
            print(f"-> {emotion}: {score:.2f}")
        
        # Display inference times
        print("\nInference Times:")
        print("-----------------")
        for operation, time_taken in result['inference_times'].items():
            print(f"-> {operation}: {time_taken:.4f} seconds")

if __name__ == "__main__":
    main()

[Gramformer] Grammar error correct/highlight model loaded..
Loading emotion classification model...


Device set to use cpu


Loading specialized fact-opinion classification model...


Device set to use cpu


Text Analysis Tool
------------------
Enter a sentence to analyze (or 'quit' to exit):

Analysis Results:
-----------------
Corrected: Nicole rides her bike.
---
Sentence type: affirmative
Classification: fact (confidence: 1.00)
Emotion detected: joy (confidence: 0.85)

All detected emotions and their confidence scores:
-> joy: 0.85
-> optimism: 0.43
-> love: 0.08
-> anticipation: 0.08
-> sadness: 0.07
-> trust: 0.02
-> pessimism: 0.02
-> disgust: 0.02
-> surprise: 0.01
-> anger: 0.01
-> fear: 0.01

Inference Times:
-----------------
-> spell_correction: 0.0000 seconds
-> grammar_correction: 0.9778 seconds
-> total_correction: 0.9778 seconds
-> sentence_type_analysis: 0.0104 seconds
-> emotion_detection: 0.1546 seconds
-> fact_opinion_classification: 0.1409 seconds
-> total_analysis: 1.2836 seconds
